In [1]:
import scipy.io as sio
import pandas as pd
import numpy as np
import scipy as scipy
import os
import h5py

In [2]:
!export CUDA_VISIBLE_DEVICES=1

In [4]:
import concise.layers as cl
from keras.models import Model, load_model
import keras.layers as kl
from keras import regularizers
import keras.optimizers as ko
from keras.callbacks import EarlyStopping
from pylab import find
from keras.models import model_from_json

from keras import backend as K

from concise.preprocessing import encodeDNA

Using TensorFlow backend.


In [1]:
import sys
sys.path.append('../helper')
import common as cm

Using TensorFlow backend.


## Summary
- Create model based on training dataset
- Evaluate model using test dataset
- Use fit_generator library of Keras to fit model with large dataset

## Helper functions

In [8]:
def get_model_sm( train_data,
              filters=64, # use powers of 2 - (32, 64, 128)
              motif_width=4, # use odd numbers - 11 or 15
              use_1by1_conv=False, # hp.choice(True, False(
              # regularization
              l1=1e-06, l1_1 = 1e-08, l1_2 = 1e-9, l1_3 = 1e-10,  
              use_two_random_seq = True,
              first_layer_units = 128, # (64, 128, 256, 512)
              lr=0.001, useIndex = False, noDense = False):
    inputs = []
    convolute = cl.ConvDNA(filters=filters, 
                   kernel_size=motif_width, activation="relu")
    convolute1by1 = kl.Conv1D(filters=filters,
                       kernel_size=1, activation='relu')
    
    dense = kl.Dense(units=1,
                     kernel_regularizer=regularizers.l1(l1))
    
    dense1 = kl.Dense(units=1,
                     kernel_regularizer=regularizers.l1(l1_1))
        
    dense2 = kl.Dense(units=1,
                     kernel_regularizer=regularizers.l1(l1_2))
    
    dense3 = kl.Dense(units=1,
                     kernel_regularizer=regularizers.l1(l1_3))
    
    inputs = [kl.Input(shape=(train_data[0]['seq'].shape[2],train_data[0]['seq'].shape[3]),
                      name='seq' + str(i)) for i in range(train_data[0]['seq'].shape[1])]
    

    #b = kl.Lambda(slice)(nw_inputs)
    
    #print(b)
    x = []
    for i in range(train_data[0]['seq'].shape[1]):
        input_dna = inputs[i]
        xi = convolute(input_dna)
        xi = convolute1by1(xi)
        xi = kl.Flatten()(xi)
        xi = dense(xi)
        
        if i==0 or i==28 or i == 56:
            xi = kl.Dense(units=1,
                     kernel_regularizer=regularizers.l1(l1_1))(xi)
        elif i < 28:
            xi = kl.Dense(units=1,
                     kernel_regularizer=regularizers.l1(l1_2))(xi)
        else:
             xi = kl.Dense(units=1,
                     kernel_regularizer=regularizers.l1(l1_3))(xi)
            

        #xi = kl.Dense(units=1)(xi) 
        #xi = kl.Activation('sigmoid')(xi) 
        
        x = x + [xi]
    
    #print(x)
    x = kl.Concatenate(axis = 1)(x)
    x = kl.Activation("softmax")(x)
    
    model = Model(inputs=inputs, outputs=x)
    
    model.compile(optimizer=ko.Adam(lr=lr), loss="kullback_leibler_divergence", metrics=[cm.r2_score_k])
    #print(model.summary())
    return model

In [27]:
def train_model_fit_generator(model, train_data_file, val_data_file, batch_size=256):
    """ Train the video classification model
    """
    with h5py.File(train_data_file, "r") as train_data, h5py.File(val_data_file, "r") as val_data:
        sample_count = int(train_data.attrs["sample_count"])
        sample_idxs = range(0, sample_count)
        training_sample_idxs = np.random.permutation(sample_idxs)
 
        sample_count = int(val_data.attrs["sample_count"])
        sample_idxs = range(0, sample_count)
        validation_sample_idxs = np.random.permutation(sample_idxs)
        
        training_sequence_generator = generate_training_sequences(batch_size,
                                                                   train_data,
                                                                   training_sample_idxs)
        validation_sequence_generator = generate_validation_sequences(batch_size,
                                                                       val_data,
                                                                       validation_sample_idxs)
        validation_steps=int(len(validation_sample_idxs)/batch_size)
        #model.fit_generator(generator=training_sequence_generator,
        #                     validation_data=validation_sequence_generator,
        #                     samples_per_epoch=len(training_sample_idxs),
        #                     nb_val_samples=len(validation_sample_idxs),
        #                     nb_epoch=100,
        #                     max_q_size=1,
        #                     verbose=2,
        #                     callbacks=[EarlyStopping(patience=5)],
        #                     class_weight=None,
        #                     nb_worker=5)
        model.fit_generator(generator=training_sequence_generator,
                            steps_per_epoch = int(len(training_sample_idxs)/batch_size),
                            epochs=300, 
                            callbacks=[EarlyStopping(patience=3)],
                            validation_data=validation_sequence_generator,
                            validation_steps=validation_steps,
                            workers=1)    

def generate_training_sequences(batch_size, train_data, training_sample_idxs):
    """ Generates training sequences on demand
    """
    while True:
        # generate sequences for training
        training_sample_count = len(training_sample_idxs)
        batches = int(training_sample_count/batch_size)
        remainder_samples = training_sample_count%batch_size
        if remainder_samples:
            batches = batches + 1
        # generate batches of samples
        for idx in range(0, batches):
            if idx == batches - 1:
                batch_idxs = training_sample_idxs[idx*batch_size:]
            else:
                batch_idxs = training_sample_idxs[idx*batch_size:idx*batch_size+batch_size]
            batch_idxs = sorted(batch_idxs)

            X = train_data["X"][batch_idxs]
            Y = train_data["Y"][batch_idxs]
            
            #X, Y = get_data(X, Y)
            yield get_data(X, Y)

def generate_validation_sequences(batch_size, val_data, validation_sample_idxs):
    """ Generates validation sequences on demand
    """
    while True:
        # generate sequences for validation
        validation_sample_count = len(validation_sample_idxs)
        batches = int(validation_sample_count/batch_size)
        remainder_samples = validation_sample_count%batch_size
        if remainder_samples:
            batches = batches + 1
        # generate batches of samples
        for idx in range(0, batches):
            if idx == batches - 1:
                batch_idxs = validation_sample_idxs[idx*batch_size:]
            else:
                batch_idxs = validation_sample_idxs[idx*batch_size:idx*batch_size+batch_size]
            batch_idxs = sorted(batch_idxs)

            X = val_data["X"][batch_idxs]
            Y = val_data["Y"][batch_idxs]

            X, Y = get_data(X, Y)
            yield X, Y

In [28]:
def train_model():
    with h5py.File("train_data_full.hdf5", "r") as train_data:
        XSample = train_data["X"][:2]
        YSample = train_data["Y"][:2]

    trainSample = get_data(XSample, YSample)
    model = get_model_sm(trainSample,             
                  filters=128, # use powers of 2 - (32, 64, 128)
                  motif_width=15, # use odd numbers - 11 or 15
                  use_1by1_conv=True,                     
                  l1=1e-08, l1_1 = 1e-08, l1_2 = 1e-10, l1_3 = 1e-12, 
                  lr=0.0001, noDense = True         

    )

    train_model_fit_generator(model, "train_data_full.hdf5", "val_data_full.hdf5")
    return model

def eval_model(model):
    with h5py.File("test_data_full.hdf5", "r") as test_data:
        XTest = test_data["X"][:]
        YTest = test_data["Y"][:]

    XTest, YTest =  get_data(XTest, YTest)
    ypredict = model.predict(XTest)

    print("SD1 " + str(scipy.stats.pearsonr(ypredict[:,0].ravel(), YTest[:,0])[0]**2))
    print("SD2 " + str(scipy.stats.pearsonr(ypredict[:,28].ravel(), YTest[:,28])[0]**2))
    print("SDCrypt " + str(scipy.stats.pearsonr(ypredict[:,55], YTest[:,55])[0]**2))
    print("SDNew " + str(scipy.stats.pearsonr((np.sum(ypredict[:,1:28], axis = 1) + np.sum(ypredict[:,29:54], axis = 1)).ravel()
                          , (np.sum(YTest[:,1:28], axis = 1) + np.sum(YTest[:,29:54], axis = 1)).ravel())[0]**2))


## Build model to predict

In [5]:
#create folder to save results
resultsdir = cm.create_folder(os.path.abspath('../results'))

In [29]:
model = train_model()

Tensor("activation_6_target:0", shape=(?, ?), dtype=float32)
Epoch 1/300
585/585 [==============================] - 180s - loss: 1.5894 - r2_score_k: 0.3928 - val_loss: 1.1129 - val_r2_score_k: 0.5176
Epoch 2/300
585/585 [==============================] - 181s - loss: 1.0819 - r2_score_k: 0.5340 - val_loss: 1.0650 - val_r2_score_k: 0.5463
Epoch 3/300
585/585 [==============================] - 176s - loss: 1.0463 - r2_score_k: 0.5540 - val_loss: 1.0357 - val_r2_score_k: 0.5612
Epoch 4/300
585/585 [==============================] - 177s - loss: 1.0172 - r2_score_k: 0.5684 - val_loss: 1.0036 - val_r2_score_k: 0.5777
Epoch 5/300
585/585 [==============================] - 174s - loss: 0.9829 - r2_score_k: 0.5863 - val_loss: 0.9651 - val_r2_score_k: 0.5976
Epoch 6/300
585/585 [==============================] - 177s - loss: 0.9435 - r2_score_k: 0.6063 - val_loss: 0.9237 - val_r2_score_k: 0.6168
Epoch 7/300
585/585 [==============================] - 179s - loss: 0.9066 - r2_score_k: 0.6229 - v

In [30]:
eval_model(model)

SD1 0.752808820729
SD2 0.751343600989
SDCrypt 0.582268528334
SDNew 0.819467765646


In [ ]:
#save model to file to retreive later
model.save(resultsdir + "model_seq_57_full.h5")